In [22]:
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import math

In [2]:
API_KEY = "ea9637fd9f0c41f3e2e932faa99dfcd76f8041aa"

In [3]:
def fetch_state_data():
    """Fetch state-level Census data, return as DataFrame."""
    state_url = (
        f"https://api.census.gov/data/2019/pep/population"
        f"?get=NAME,POP&for=state:*&key={API_KEY}"
    )
    r_states = requests.get(state_url)
    state_data = r_states.json()
    df = pd.DataFrame(state_data[1:], columns=state_data[0])
    df["POP"] = df["POP"].astype(int)
    return df

In [4]:
def fetch_county_data():
    """Fetch county-level Census data, return as DataFrame."""
    county_url = (
        f"https://api.census.gov/data/2019/pep/population"
        f"?get=NAME,POP&for=county:*&key={API_KEY}"
    )
    r_counties = requests.get(county_url)
    county_data = r_counties.json()
    df = pd.DataFrame(county_data[1:], columns=county_data[0])
    df["POP"] = df["POP"].astype(int)
    # Split the NAME column into state and county
    df[['countyName', 'stateName']] = df['NAME'].str.split(',', expand=True)
    # Combine state and county to form FIPS
    df["FIPS"] = df["state"].str.zfill(2) + df["county"].str.zfill(3)
    return df

In [5]:
def fetch_geojson(url: str):
    """Cache the geojson data from the provided URL."""
    return requests.get(url).json()

In [6]:
def build_states_gdf(state_df, state_abbrev_to_fips):
    """Build GeoDataFrame for US states."""
    state_pop_dict = state_df.set_index("state")["POP"].to_dict()
    state_name_dict = state_df.set_index("state")["NAME"].to_dict()
    url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
    geo_data = fetch_geojson(url)
    rows = []
    for feat in geo_data["features"]:
        abbrev = feat["id"]
        geom = shape(feat["geometry"])
        fips = state_abbrev_to_fips.get(abbrev)
        if fips:
            pop_val = state_pop_dict.get(fips, "No data")
            name_val = state_name_dict.get(fips, "No data")
            rows.append({
                "geometry": geom,
                "STATE_FIPS": fips,
                "NAME": name_val,
                "POP": pop_val
            })
    return gpd.GeoDataFrame(rows, crs="EPSG:4326")

In [7]:
def build_counties_gdf(county_df):
    """Build GeoDataFrame for US counties and simplify geometries for performance."""
    county_pop_dict = county_df.set_index("FIPS")["POP"].to_dict()
    county_name_dict = county_df.set_index("FIPS")["NAME"].to_dict()
    url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
    geo_data = fetch_geojson(url)
    rows = []
    for feat in geo_data["features"]:
        fips = feat["id"]
        geom = shape(feat["geometry"])
        pop_val = county_pop_dict.get(fips, "No data")
        name_val = county_name_dict.get(fips, "No data")
        rows.append({
            "geometry": geom,
            "FIPS": fips,
            "NAME": name_val,
            "POP": pop_val
        })
    gdf = gpd.GeoDataFrame(rows, crs="EPSG:4326")
    gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.01, preserve_topology=True)
    return gdf

In [8]:
state_abbrev_to_fips = {
    "AL": "01", "AK": "02", "AZ": "04", "AR": "05", "CA": "06", "CO": "08",
    "CT": "09", "DE": "10", "DC": "11", "FL": "12", "GA": "13", "HI": "15",
    "ID": "16", "IL": "17", "IN": "18", "IA": "19", "KS": "20", "KY": "21",
    "LA": "22", "ME": "23", "MD": "24", "MA": "25", "MI": "26", "MN": "27",
    "MS": "28", "MO": "29", "MT": "30", "NE": "31", "NV": "32", "NH": "33",
    "NJ": "34", "NM": "35", "NY": "36", "NC": "37", "ND": "38", "OH": "39",
    "OK": "40", "OR": "41", "PA": "42", "RI": "44", "SC": "45", "SD": "46",
    "TN": "47", "TX": "48", "UT": "49", "VT": "50", "VA": "51", "WA": "53",
    "WV": "54", "WI": "55", "WY": "56"
}

In [9]:
def load_and_merge_caps(_states_gdf):
    caps_df = pd.read_csv("caps_plans.csv")
    caps_df["State"] = caps_df["State"].str.strip().str.upper()
    caps_df["STATE_FIPS"] = caps_df["State"].map(state_abbrev_to_fips)
    caps_df["plan_info"] = caps_df.apply(
        lambda row: f"{row['City']}, {row['Year']}, {row['Plan Type']}", axis=1
    )
    grouped = caps_df.groupby("STATE_FIPS").agg(
        n_caps=("Plan Type", "count"),
        plan_list=("plan_info", lambda x: list(x))
    ).reset_index()
    merged = _states_gdf.merge(grouped, on="STATE_FIPS", how="left")
    merged["n_caps"] = merged["n_caps"].fillna(0).astype(int)
    merged["plan_list"] = merged["plan_list"].apply(lambda x: x if isinstance(x, list) else [])
    return merged

In [10]:
def load_and_merge_caps_county(_counties_gdf):
    caps_df = pd.read_csv("caps_plans.csv")
    mapping_df = pd.read_csv("city_county_mapping.csv")
    # Standardize text for matching
    caps_df["State"] = caps_df["State"].str.strip().str.upper()
    mapping_df["CountyKey"] = mapping_df["CountyName"].apply(
        lambda x: x.upper().split(',')[0].replace(" COUNTY", "").strip()
    )
    merged_caps = pd.merge(
        caps_df, mapping_df, 
        left_on=["City", "State"], 
        right_on=["CityName", "StateName"], 
        how="left"
    )
    merged_caps["plan_info"] = merged_caps.apply(
        lambda row: f"{row['City']}, {row['Year']}, {row['Plan Type']}", axis=1
    )
    merged_caps["CountyKey"] = merged_caps["CountyName"].apply(
        lambda x: x.upper().split(',')[0].replace(" COUNTY", "").strip() if pd.notnull(x) else None
    )
    grouped = merged_caps.groupby(["CountyKey", "StateName"]).agg(
        n_caps=("Plan Type", "count"),
        plan_list=("plan_info", lambda x: list(x))
    ).reset_index()
    fips_to_abbrev = {v: k for k, v in state_abbrev_to_fips.items()}
    _counties_gdf["STATE"] = _counties_gdf["FIPS"].str[:2].map(fips_to_abbrev)
    _counties_gdf["CountyKey"] = _counties_gdf["NAME"].apply(
        lambda x: x.upper().split(',')[0].replace(" COUNTY", "").strip()
    )
    merged_counties = _counties_gdf.merge(
        grouped, 
        left_on=["CountyKey", "STATE"], 
        right_on=["CountyKey", "StateName"], 
        how="left"
    )
    merged_counties["n_caps"] = merged_counties["n_caps"].fillna(0).astype(int)
    merged_counties["plan_list"] = merged_counties["plan_list"].apply(lambda x: x if isinstance(x, list) else [])
    return merged_counties

In [11]:
def load_city_mapping():
    """Load the city mapping CSV for marker locations."""
    df = pd.read_csv("city_county_mapping.csv")
    df["CityName"] = df["CityName"].str.strip().str.upper()
    df["StateName"] = df["StateName"].str.strip().str.upper()
    df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
    df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
    return df

In [12]:
def load_city_plans():
    """Load and group caps_plans data by city and state."""
    df = pd.read_csv("caps_plans.csv")
    df["City"] = df["City"].str.strip().str.upper()
    df["State"] = df["State"].str.strip().str.upper()
    df["plan_info"] = df.apply(lambda row: f"{row['Year']}, {row['Plan Type']}", axis=1)
    grouped = df.groupby(["City", "State"]).agg(plan_list=("plan_info", lambda x: list(x))).reset_index()
    return grouped

In [16]:
state_df = fetch_state_data()
county_df = fetch_county_data()

states_gdf = build_states_gdf(state_df, state_abbrev_to_fips)
counties_gdf = build_counties_gdf(county_df)

In [17]:
state_df.head()

,NAME,POP,state
0,Alabama,4903185,01
1,Alaska,731545,02
2,Arizona,7278717,04
3,Arkansas,3017804,05
4,California,39512223,06


In [18]:
county_df.head()

,NAME,POP,state,county,countyName,stateName,FIPS
0,"Fayette County, Illinois",21336,17,051,Fayette County,Illinois,17051
1,"Logan County, Illinois",28618,17,107,Logan County,Illinois,17107
2,"Saline County, Illinois",23491,17,165,Saline County,Illinois,17165
3,"Lake County, Illinois",696535,17,097,Lake County,Illinois,17097
4,"Massac County, Illinois",13772,17,127,Massac County,Illinois,17127


In [19]:
states_gdf.head()

,geometry,STATE_FIPS,NAME,POP
0,"POLYGON ((-87.3593 35.00118, -85.60668 34.9847...",01,Alabama,4903185
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",02,Alaska,731545
2,"POLYGON ((-109.0425 37.00026, -109.04798 31.33...",04,Arizona,7278717
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",05,Arkansas,3017804
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",06,California,39512223


In [20]:
counties_gdf.head()

,geometry,FIPS,NAME,POP
0,"POLYGON ((-86.49677 32.34444, -86.7179 32.4028...",01001,"Autauga County, Alabama",55869
1,"POLYGON ((-86.5778 33.76532, -86.75914 33.8406...",01009,"Blount County, Alabama",57826
2,"POLYGON ((-85.18413 32.87052, -85.12342 32.772...",01017,"Chambers County, Alabama",33254
3,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",01021,"Chilton County, Alabama",44428
4,"POLYGON ((-88.13999 34.5817, -88.09789 34.8922...",01033,"Colbert County, Alabama",55241


In [25]:
states_gdf_caps =load_and_merge_caps(states_gdf)
states_gdf_caps.head()

,geometry,STATE_FIPS,NAME,POP,n_caps,plan_list
0,"POLYGON ((-87.3593 35.00118, -85.60668 34.9847...",01,Alabama,4903185,1,"[Birmingham, 2013, Green Plan]"
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",02,Alaska,731545,1,"[Anchorage, 2019, Mitigation Primary CAP]"
2,"POLYGON ((-109.0425 37.00026, -109.04798 31.33...",04,Arizona,7278717,2,"[Glendale, 2011, Green Plan, Phoenix, 2021, Mi..."
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",05,Arkansas,3017804,0,[]
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",06,California,39512223,7,"[Oakland, 2020, Mitigation Primary CAP, Los An..."


In [35]:
counties_gdf_caps = load_and_merge_caps_county(counties_gdf)
counties_gdf_caps.head()

,geometry,FIPS,NAME,POP,STATE,CountyKey,StateName,n_caps,plan_list
0,"POLYGON ((-86.49677 32.34444, -86.7179 32.4028...",01001,"Autauga County, Alabama",55869,AL,AUTAUGA,NaN,0,[]
1,"POLYGON ((-86.5778 33.76532, -86.75914 33.8406...",01009,"Blount County, Alabama",57826,AL,BLOUNT,NaN,0,[]
2,"POLYGON ((-85.18413 32.87052, -85.12342 32.772...",01017,"Chambers County, Alabama",33254,AL,CHAMBERS,NaN,0,[]
3,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",01021,"Chilton County, Alabama",44428,AL,CHILTON,NaN,0,[]
4,"POLYGON ((-88.13999 34.5817, -88.09789 34.8922...",01033,"Colbert County, Alabama",55241,AL,COLBERT,NaN,0,[]


In [36]:
city_mapping_df = load_city_mapping()
city_plans_df = load_city_plans()

In [37]:
city_mapping_df.head()

,CityName,StateName,CountyName,Latitude,Longitude
0,ALAMEDA,CA,"Alameda County, California",37.7652,-122.2416
1,ALBUQUERQUE,NM,"Bernalillo County, New Mexico",35.0844,-106.6504
2,ALEXANDRIA,VA,"Alexandria City, Virginia",38.8048,-77.0469
3,ANAHEIM,CA,"Orange County, California",33.8366,-117.9143
4,ANCHORAGE,AK,"Anchorage Municipality, Alaska",61.2181,-149.9003


In [38]:
city_plans_df.head()

,City,State,plan_list
0,ALAMEDA,CA,"[2019, Equal Adaptation-Mitigation Plan]"
1,ALBUQUERQUE,NM,"[2021, Green Plan]"
2,ALEXANDRIA,VA,"[2019, Mitigation Only CAP]"
3,ANAHEIM,CA,"[2020, Mitigation Plan]"
4,ANCHORAGE,AK,"[2019, Mitigation Primary CAP]"


In [41]:
counties_gdf_caps.to_csv("counties_gdf.csv", index=False)

In [42]:
pd.read_csv("counties_gdf.csv")

,geometry,FIPS,NAME,POP,STATE,CountyKey,StateName,n_caps,plan_list
0,"POLYGON ((-86.496774 32.344437, -86.717897 32....",1001,"Autauga County, Alabama",55869,AL,AUTAUGA,NaN,0,[]
1,"POLYGON ((-86.577799 33.765316, -86.759144 33....",1009,"Blount County, Alabama",57826,AL,BLOUNT,NaN,0,[]
2,"POLYGON ((-85.184131 32.870525, -85.123421 32....",1017,"Chambers County, Alabama",33254,AL,CHAMBERS,NaN,0,[]
3,"POLYGON ((-86.517344 33.020566, -86.515959 32....",1021,"Chilton County, Alabama",44428,AL,CHILTON,NaN,0,[]
4,"POLYGON ((-88.139988 34.581703, -88.097888 34....",1033,"Colbert County, Alabama",55241,AL,COLBERT,NaN,0,[]
...,...,...,...,...,...,...,...,...,...
3216,"MULTIPOLYGON (((-75.242266 38.027209, -75.3590...",51001,"Accomack County, Virginia",32316,VA,ACCOMACK,NaN,0,[]
3217,"POLYGON ((-81.225104 37.234874, -81.112596 37....",51021,"Bland County, Virginia",6280,VA,BLAND,NaN,0,[]
3218,"POLYGON ((-81.968297 37.537798, -81.92787 37.5...",51027,"Buchanan County, Virginia",21004,VA,BUCHANAN,NaN,0,[]
3219,"POLYGON ((-78.443319 37.0794, -78.493028 36.89...",51037,"Charlotte County, Virginia",11880,VA,CHARLOTTE,NaN,0,[]
